# Introduction
Imagine your company uses a server that runs a service called ticky, an internal ticketing system. The service logs events to syslog, both when it runs successfully and when it encounters errors.

The service's developers need your help getting some information from those logs so that they can better understand how their software is used and how to improve it. So, for this lab, you'll write some automation scripts that will process the system log and generate reports based on the information extracted from the log files.

What you'll do
- Use regex to parse a log file

- Append and modify values in a dictionary

- Write to a file in CSV format

- Move files to the appropriate directory for use with the CSV->HTML converter

# Generate reports
Now, we're going to practice creating a script, named ticky_check.py, that generates two different reports from this internal ticketing system log file i.e., syslog.log. This script will create the following reports:

- The ranking of errors generated by the system: A list of all the error messages logged and how many times each error was found, sorted by the most common error to the least common error. This report doesn't take into account the users involved.

- The user usage statistics for the service: A list of all users that have used the system, including how many info messages and how many error messages they've generated. This report is sorted by username.
To create these reports write a python script named ticky_check.py. Use nano editor for this.

In [1]:
# nano ticky_check.py
#!/usr/bin/env python3

In [ ]:
import re
import operator
import csv
import sys

error = {}
per_user = {}

with open("syslog.log") as log:
   for line in log:
      user = re.search(r"\((.*)\)",line).group(1)
      counter_user = {"INFO" : 0, "USER" : 0}
      if user not in per_user:
         per_user[user] = counter_user

      if "INFO" in line:
         per_user[user]["INFO"] += 1

      elif "ERROR" in line:
         error_log = re.search(r"ERROR (.*) ",line).group(1)
         if error_log not in error:
            error[error_log]= 0
         error[error_log] += 1
         per_user[user]['ERROR'] += 1

sort_error=[]
sort_user=[]

for error, count in sorted(error.items(), key=lambda item: item[1],reverse=True):
   sort_error.append([error, count])
for user in sorted(per_user.keys()):
   sort_user.append([user, per_user[user]["INFO"], per_user[user]["ERROR"]])


sort_error.insert(0,["Error","Count"])
sort_user.insert(0,["Username","INFO","ERROR"])

with open("error_message.csv","w") as error_csv:
   writer = csv.writer(error_csv)
   writer.writerows(sort_error)
   error_csv.close()
with open("user_statistics.csv","w") as user_stat:
   writer = csv.writer(user_stat)
   writer.writerows(sort_user)
   user_stat.close()


In [ ]:
#!/usr/bin/env python3
import re
import csv
import operator

error = {}
per_user = {}
sys_log =r"/home/student-00-0322d63a553d/syslog.log"
pattern = r"(INFO|ERROR) ([\w' ]+|[\w\[\]#' ]+) (\(\w+\)|\(\w+\.\w+\))$"

with open(sys_log, "r") as f:
 for line in f:
  search_res = re.search(pattern, line)
  if search_res is None:
   continue
  if search_res.groups()[0] == "INFO":
   cat_msg = search_res.groups()[0]
   message_res = search_res.groups()[1]
   name_res = str(search_res.groups()[2])[1:-1]
   if name_res in per_user:
    user = per_user[name_res]
    user[cat_msg] += 1
   else:
    per_user[name_res] = {'INFO':1, 'ERROR':0}
  if search_res.groups()[0] == "ERROR":
   cat_msg = search_res.groups()[0]
   message_res = search_res.groups()[1]
   name_res = str(search_res.groups()[2])[1:-1]
   error[message_res] = error.get(message_res, 0) + 1
   if name_res in per_user:
    user = per_user[name_res]
    user[cat_msg] += 1
   else:
    per_user[name_res] = {'INFO':0, 'ERROR':1}

sort_msg = [("Error", "Count")] + sorted(error.items(), key = operator.itemgetter(1), reverse=True)
sort_user= [("student-00-0322d63a553d", "INFO", "ERROR")] + sorted(per_user.items())[0:8]


with open("error_message.csv", "w") as error_msg:
 for line in sort_msg:
  error_msg.write("{}, {}\n".format(line[0], line[1]))

with open("user_statistics.csv", "w") as user_stat:
 for line in sort_user:
  if isinstance(line[1], dict):
   user_stat.write("{}, {}, {}\n".format(line[0], line[1].get("INFO"), line[1].get("ERROR")))
  else:
   user_stat.write("{}, {}, {}\n".format(line[0], line[1], line[2]))



In [ ]:
./csv_to_html.py error_message.csv /var/www/html/html_error.html

./csv_to_html.py user_statistics.csv /var/www/html/html_stat.html